# 1. Description of Setups

For each sample size:

* 1000 datasets:
    * $T = \left\{100, 200, 400, 800, 1600, 3200\right\}$ independent markets per dataset
    * $J = 25$ products per market
        * Each product $j$ in each market $t$ has an observed characteristic vector $x_{jt} = (1, p_{jt}, d_{jt},  x_{jt}^1,  x_{jt}^2, \xi_{jt}, z_{jt})$
            * $p_{jt} \sim Lognormal(0, 1)$: continuous "price" variable; lognormal distribution roughly mimics empirical distribution of prices
            * $d_{jt}$: binary variable (group/nest 0 or 1)
            * $x_{jt}^1, x_{jt}^2 \sim U[0.1,5]$: continuous covariates
            * $\xi_{jt} \sim N(0,1)$: unobserved product characteristic, correlated with the price
            * $z_{jt} \sim N(0,1)$: a simulated instrument

Treat $d_{jt}$ as the realization of a latent continuous variable $d_{jt}^*$, with $ d_{jt} = 1\left\{d_{jt}^* > 0\right\}$. Furthermore, $x_{jt}^i$, for $i=1,2$, is constructed by evenly positioning the instances of the latent variable $(x_{jt}^i)^*$ on the interval $[0.1,5].$ Specifically, assume:
$$  \begin{pmatrix} \ln p \\ d^* \\ (x^1)^* \\  (x^2)^* \\ \xi \\ z \end{pmatrix}
    \sim
    N
    \begin{pmatrix} \begin{matrix} 0 \\ 0 \\ 0 \\ 0 \\ 0 \\ 0 \end{matrix},
    \begin{matrix}  1 & \sigma & \sigma & \sigma & \sigma & \sigma \\
                    \sigma & 1 & 0 & 0 & 0 & 0 \\
                    \sigma & 0 & 1 & 0 & 0 & 0 \\
                    \sigma & 0 & 0 & 1 & 0 & 0 \\
                    \sigma & 0 & 0 & 0 & 1 & 0 \\
                    \sigma & 0 & 0 & 0 & 0 & 1 \\\end{matrix} \end{pmatrix} $$.

Preference parameters:

* Utility weights on the price $\alpha$ and on the remaining covariates and their interactions $\beta = (\beta_0, \beta_d, \beta_{x^1}, \beta_{x^2}, \beta_{dx^1}, \beta_{dx^2})$
* Nesting parameter $\rho$, reflecting consumer heterogeneity in preferences for $d$

True DGP: Indirect utility of consumer $i$ from product $j$ of nest $g$ in market $t$:

$$ v_{ijt} = \beta_0 + \alpha p_{jt} + \beta_d d_{jt} + \beta_{x^1} \ln{x_{jt}^1} + \beta_{x^2} \ln{x_{jt}^2} + \beta_{dx^1} d_{jt} \ln{x_{jt}^1} + \beta_{dx^2} d \ln{x_{jt}^2} + \sum_g{I\left\{d_{jt} = g\right\} \zeta_{ig}} + (1 - \rho) \epsilon_{ijt} $$
where

* $I(d = g)$ is an indicator function that equals 1 when $d = g$
* $\zeta_{ig}$ is an individual-specific taste parameter common for consumer $i$ to all products in nest $g$
* $\epsilon_{ijt}$ is an individual-specific taste parameter for product $j$ that is i.i.d. Type-I Extreme Value

Estimated function derived from the nested logit model:

$$ \ln{s_{jt}} - \ln{s_{0t}} = \alpha p_{jt} + f(d_{jt}, x_{jt}^1, x_{jt}^2) + \rho \ln{s_{j/g,t}} + \xi_{jt} $$
where
* $s_{jt}$ is the market share of product $j$ in market $t$
* $s_{0t}$ is the outside option share in market $t$
* $s_{j/g,t}$ is the within-nest share of product $j$ in nest $g$, in market $t$

# 2. Setup: Defining the DGP and Estimation Functions

Libraries and setting seed:

In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm            # for OLS
from linearmodels import IV2SLS         # for 2SLS
import pyblp                            # for differentiation instruments
from econml.grf import CausalForest     # for causal forest
import keras                            # for NNet
from econml.iv.dml import DMLIV         # for DML with IV
from econml.dml import LinearDML        # for DML
import xgboost as xgb                   # for DML orthogonalization
import time
import os
import functools


%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (18, 12)

seed = 1000

np.random.seed(seed)

Timer function as a decorator for the estimations:

In [2]:
def timer(timed_function):
    @functools.wraps(timed_function) # just to have func.__name__ return the function name rather than "wrapper"
    def wrapper(*args, **kwargs):
        t1 = time.perf_counter()
        result = timed_function(*args, **kwargs)
        print(f'{timed_function.__name__} ({round(time.perf_counter() - t1, 2)})', end = ' ')
        return result
    return wrapper

## 2.1. Defining the data generating process and its parameters

In [3]:
# T = insert number
class LogitSim:
    """Class for simulating the data"""
    
    
    # Class Variables ----------------------------------------------------------------------------------------------------
    
    T = #SET QUANTITY TO RUN # number of markets
    J = 25                   # number of products per market
    beta = np.array([-3, -2, -1, 1, 1, 1, 1])  # utility parameters (including alpha)
    rho = 0.3                # nesting parameter
    z = 1      # number of instruments
    c = 0.3    # correlation parameter in the DGP covariance matrix
    
    # Initializer --------------------------------------------------------------------------------------------------------
    #    1. Product features
    #    2. Resulting market shares and other NL estimation statistics
    #    3. Resulting elasticities
    
    def __init__(self):
        """Initializer of the features data, including their market shares and
        their own and cross price elasticities.
        Returned attributes:
            * sj, ln_sj: market share and its log
            * ln_s0: outside option log market share
            * ln_sjg: log within-nest market share
            * y: NL outcome
            * X: NL covariate matrix
            * Z: excluded instruments matrix
            * d_index: indicator matrix – 1 when d_i = d_j, 0 when d_i != d_j
            * elasticities: true elasticity matrix
        """
        
        T = self.T
        J = self.J
        beta = self.beta
        rho = self.rho
        z = self.z
        c = self.c
        
        ###########################
        #        Features
        ###########################
        p = 5 + z    # dimension of the DGP covariance matrix
        covMat = np.random.multivariate_normal(mean = np.repeat(0, p),
                                               cov = (np.identity(p) +
                                                      np.hstack((0,
                                                                 np.repeat(c, p-1),
                                                                 np.repeat(0, (p-1)*p))).reshape(p,p) +
                                                      np.hstack((0,
                                                                 np.repeat(c, p-1),
                                                                 np.repeat(0, (p-1)*p))).reshape(p,p).T),
                                               size = T*J)
        features = np.column_stack((
            np.ones(T*J),                                                # constant
            np.exp(covMat[:,0]),                                         # p
            (covMat[:,1] > 0).astype(int),                               # d
            np.log(((covMat[:,2].argsort().argsort()) / ((T*J - 1) / 4.9)) + 0.1),     # ln(x1) (evenly distributed on [0.1,5])
            np.log(((covMat[:,3].argsort().argsort()) / ((T*J - 1) / 4.9)) + 0.1),     # ln(x2) (evenly distributed on [0.1,5])
            (covMat[:,1] > 0).astype(int) * np.log(((covMat[:,2].argsort().argsort()) /
                                                    ((T*J - 1) / 4.9)) + 0.1),    # d*ln(x1)
            (covMat[:,1] > 0).astype(int) * np.log(((covMat[:,3].argsort().argsort()) /
                                                    ((T*J - 1) / 4.9)) + 0.1),    # d*ln(x2)
            covMat[:,4:p]                                                # xi, z
        ))
        
        #############################################################
        #      Market shares and other NL estimation statistics
        #############################################################
        # 1. Utility net of epsilon – dimensions (TxJ, N)
        numerators = np.exp((features[:,0:7] @ beta.T + features[:,7]) / (1 - rho))

        # 2. Computing denominators
        D0 = np.repeat(np.sum((numerators * (features[:,2] == 0)).reshape(T,J), axis = 1), repeats = J, axis = 0)
        D1 = np.repeat(np.sum((numerators * (features[:,2] == 1)).reshape(T,J), axis = 1), repeats = J, axis = 0)
        Dg = np.where(features[:,2] == 1, D1, D0)
        D  = D0**(1-rho) + D1**(1-rho) + 1
        denominators = (Dg**rho) * D

        # 3. Dividing to get the market shares
        sj = numerators / denominators
        ln_sj = np.log(sj)

        # 4. Computing outside option share
        s0 = np.repeat(1 - np.sum(sj.reshape(T, J), axis = 1), repeats = J)
        ln_s0 = np.log(s0)

        # 5. Computing within-nest shares (the nansum is needed for cases when there is only one group in a market)
        sjg = np.nansum(
            np.column_stack((
                (sj * features[:,2]) / np.repeat(np.sum(sj.reshape(T, J) * features[:,2].reshape(T, J), axis = 1),
                                                 repeats = J),
                (sj * (1 - features[:,2])) / np.repeat(np.sum(sj.reshape(T, J) * (1 - features[:,2]).reshape(T, J), axis = 1),
                                                       repeats = J)
            )),
            axis = 1
        )
        ln_sjg = np.log(sjg)
        
        # 6. NL outcome variable, X matrix, and Z (instruments) matrix
        y = ln_sj - ln_s0
        X = np.column_stack((features[:,0:3], np.exp(features[:,3:5])))   # constant, p, d, x1, x2
        products_df = pd.DataFrame(
            np.column_stack((sj, X[:,1:5], np.repeat(np.array([range(0,T)]), J), np.arange(0, T*J))),
            columns = ['shares', 'prices', 'd', 'x1', 'x2', 'market_ids', 'firm_ids']
        )
        quadratic_instruments = pyblp.build_differentiation_instruments(
            pyblp.Formulation('prices + d + x1 + x2'),
            products_df,
            version = 'quadratic'
        )
        Z = np.column_stack((features[:,8:], quadratic_instruments[:,7:10]))
        
        ###########################
        #       Elasticities
        ###########################
        # Indicator matrix for whether the features are in the same nest
        self.d_index = (np.repeat(features[:,2].reshape(T, J), repeats = J, axis = 0) + features[:,2].reshape(T*J, 1) != 1)

        elasticities = (np.repeat((-beta[1] * sj * features[:,1]).reshape(T,J), repeats = J, axis = 0) *
                        np.select(condlist   = [self.d_index,                # same nest
                                                ~self.d_index],              # different nest
                                  choicelist = [(rho*Dg**(rho-1)*D + 1 - rho).reshape(T*J, 1) / (1 - rho), 1]) +
                        (beta[1] * features[:,1].reshape(T*J,1) / (1 - rho)) * np.tile(np.identity(J), reps = (T,1)))
        
        ############################
        #    Returned Attributes
        ############################
        self.sj = sj
        self.ln_sj = ln_sj
        self.ln_s0 = ln_s0
        self.ln_sjg = ln_sjg
        self.y = y
        self.X = X
        self.Z = Z
        self.elasticities = elasticities

## 2.2. First Step Estimators: 2SLS specifications and DMLIV

In [4]:
class FirstStepEstimates:
    """Class for conducting the first step estimations (alpha and rho values;
    in the linear cases also the remaining parameter estimates) and storing
    their outputs. 3 models:
        1. 2SLS with linear covariates
        2. 2SLS with a log transformation of x1 and x2
        3. DML-IV – parametrically estimating alpha and rho while
           controlling nonparametrically for the covariates
    """
    
    
    def __init__(self, alpha = None, rho = None, beta = None, elasticities = None, eta_summary = None):
        self.alpha = [] if alpha is None else alpha
        self.rho = [] if rho is None else rho
        self.beta = [] if beta is None else beta
        self.elasticities = elasticities
        self.eta_summary = [] if eta_summary is None else eta_summary
    
    # Static Methods -----------------------------------------------------------------------------------------------------
    
    # 1. Computing the simulated products' elasticities based on parameter estimates
    # 2. Summarizing the elasticity data:
    #        a. Computing the average own price elasticities, within-nest cross price elasticities, and the across-nest
    #           cross price elasticities
    #        b. Computing the root mean squared errors of each of these three elasticity groups
    
    @staticmethod
    def compute_elasticities(SimData, alpha, rho):
        D0 = np.repeat(np.sum((np.exp((SimData.ln_sj - SimData.ln_s0 - rho * SimData.ln_sjg) / (1 - rho)) * 
                               (SimData.X[:,2] == 0)).reshape(SimData.T, SimData.J), axis = 1),
                       repeats = SimData.J, axis = 0)
        D1 = np.repeat(np.sum((np.exp((SimData.ln_sj - SimData.ln_s0 - rho * SimData.ln_sjg) / (1 - rho)) * 
                               (SimData.X[:,2] == 1)).reshape(SimData.T, SimData.J), axis = 1),
                       repeats = SimData.J, axis = 0)
        Dg = np.where(SimData.X[:,2] == 1, D1, D0)
        D  = 1 + np.where(D0 == 0, 0, D0**(1-rho)) + np.where(D1 == 0, 0, D1**(1-rho))

        elasticities = \
            (np.repeat((-alpha * SimData.sj * SimData.X[:,1]).reshape(SimData.T, SimData.J),
                       repeats = SimData.J, axis = 0) *
                np.select(condlist   = [SimData.d_index,       # same nest
                                        ~SimData.d_index],     # different nest
                          choicelist = [(rho * Dg**(rho-1) * D + 1 - rho).reshape(SimData.T * SimData.J, 1) /(1 - rho), 1]) +
                (alpha * SimData.X[:,1].reshape(SimData.T * SimData.J, 1) / (1 - rho)) *
                np.tile(np.identity(SimData.J), reps = (SimData.T, 1)))
        return elasticities
    
    @staticmethod
    def eta_summary(SimData, eta_est):
        avg_own         = np.mean(eta_est[np.tile(np.identity(SimData.J), reps = (SimData.T,1)) == 1])
        avg_cross_same  = np.mean(eta_est[(np.tile(np.identity(SimData.J), reps = (SimData.T,1)) == 0) & (SimData.d_index == True)])
        avg_cross_diff  = np.mean(eta_est[SimData.d_index == False])
        rmse_own        = (np.mean((SimData.elasticities[np.tile(np.identity(SimData.J), reps = (SimData.T,1)) == 1] -
                                    eta_est[np.tile(np.identity(SimData.J), reps = (SimData.T,1)) == 1])**2))**.5
        rmse_cross_same = (np.mean((SimData.elasticities[(np.tile(np.identity(SimData.J), reps = (SimData.T,1)) == 0) & (SimData.d_index == True)] -
                                    eta_est[(np.tile(np.identity(SimData.J), reps = (SimData.T,1)) == 0) & (SimData.d_index == True)])**2))**.5
        rmse_cross_diff = (np.mean((SimData.elasticities[SimData.d_index == False] -
                                    eta_est[SimData.d_index == False])**2))**.5
        return np.array([avg_own, avg_cross_same, avg_cross_diff, rmse_own, rmse_cross_same, rmse_cross_diff])
    
    
    # Methods ------------------------------------------------------------------------------------------------------------
    
    # First step estimates
    #    a. Linear 2SLS
    #    b. Linear 2SLS with log transfomration
    #    c. DML-IV
    
    @timer
    def LinIV(self, SimData):
        theta = IV2SLS(dependent = SimData.y,
                       exog = np.delete(SimData.X, 1, 1),
                       endog = np.column_stack((SimData.X[:,1], SimData.ln_sjg)),
                       instruments = SimData.Z).fit().params
        self.alpha.append(theta[-2])  # endog vars always at the end
        self.rho.append(theta[-1])
        self.beta.append(theta[:-2])
        self.elasticities = FirstStepEstimates.compute_elasticities(SimData, self.alpha[-1], self.rho[-1])
        self.eta_summary.append(FirstStepEstimates.eta_summary(SimData, self.elasticities))
    
    @timer
    def LinIVLog(self, SimData):
        theta = IV2SLS(dependent = SimData.y,
                       exog = np.column_stack((SimData.X[:,[0,2]], np.log(SimData.X[:,3:5]))),
                       endog = np.column_stack((SimData.X[:,1], SimData.ln_sjg)),
                       instruments = SimData.Z).fit().params
        self.alpha.append(theta[-2])
        self.rho.append(theta[-1])
        self.beta.append(theta[:-2])
        self.elasticities = FirstStepEstimates.compute_elasticities(SimData, self.alpha[-1], self.rho[-1])
        self.eta_summary.append(FirstStepEstimates.eta_summary(SimData, self.elasticities))
    
    @timer
    def DMLIV(self, SimData,
              depth_yx, n_est_yx, lr_yx,
              depth_tx, n_est_tx, lr_tx,
              depth_txz, n_est_txz, lr_txz):
        # Initializing the estimator
        dmliv_est = DMLIV(model_y_xw  = xgb.XGBRegressor(max_depth = depth_yx, n_estimators = n_est_yx, learning_rate = lr_yx),
                          model_t_xw  = xgb.XGBRegressor(max_depth = depth_tx, n_estimators = n_est_tx, learning_rate = lr_tx),
                          model_t_xwz = xgb.XGBRegressor(max_depth = depth_txz, n_estimators = n_est_txz, learning_rate = lr_txz))
        # Fitting and storing the alpha and rho values
        dmliv_est.fit(Y = SimData.y,
                      T = np.column_stack((SimData.X[:,1], SimData.ln_sjg)),
                      Z = SimData.Z,
                      X = None,
                      W = SimData.X[:,2:5])
        alpha_, rho_ = dmliv_est.const_marginal_effect().reshape(-1)
        self.alpha.append(alpha_)
        self.rho.append(rho_)
        self.elasticities = FirstStepEstimates.compute_elasticities(SimData, self.alpha[-1], self.rho[-1])
        self.eta_summary.append(FirstStepEstimates.eta_summary(SimData, self.elasticities))

## 2.3. Second Step Estimators: DML, GRF, and MLP

In [5]:
class SecondStepEstimates:
    """Class for conducting the second step estimations and storing their
    outputs.
    This step estimates average treatment effects, conditional average
    treatment effects, and marginal effects of the covariates, assuming the
    alpha and rho values of the first step DML-IV estimation.
        1. DML: returns an ATE estimate for a selected covariate in X
        2. GRF: returns CATE estimates for a selected covariate in X (the
           implementation here uses a regular causal forest; can be replaced
           by a generalized random forest in the presence of an endogenous
           covariate of interest)
        3. MLP: returns marginal effect estimates for a selected covariate
           in X
            * Note: the MLP architecture is intentionally very wide relative
              to the complexity of the problem, with the aim of mitigating
              regularization bias caused by the early stopping rule. The
              desire to avoid regularization bias also explains the absence
              of other forms of regularization. This preference is due to
              the nature of the problem at hand, which is interested in
              quantifying a causal relationship rather than in prediction,
              and therefore places more weight on bias reduction, at the
              expense of the estimates' variance.
        """
    
    def __init__(self, effect = None):
        self.effect = [] if effect is None else effect
    
    # Methods ------------------------------------------------------------------------------------------------------------
    
    # Second step estimates
    #    a. DML (ATE)
    #    b. GRF (CATE)
    #    c. MLP (marginal effects)
    
    @timer
    def DML(self, SimData, estimates, treatment,
            depth_y, n_est_y, lr_y,
            depth_t, n_est_t, lr_t):
        # Initializing
        dml_est = LinearDML(model_y = xgb.XGBRegressor(max_depth = depth_y, n_estimators = n_est_y, learning_rate = lr_y),
                            model_t = xgb.XGBRegressor(max_depth = depth_t, n_estimators = n_est_t, learning_rate = lr_t))
        # Fitting and returning the ATE of the covariate selected as the treatment
        dml_est.fit(Y = SimData.y - SimData.ln_sjg * estimates.rho[-1] - SimData.X[:,1] * estimates.alpha[-1],
                      T = SimData.X[:, treatment],
                      X = None,
                      W = np.delete(SimData.X[:,2:5], treatment - 2, 1))
        ATE = dml_est.ate()
        self.effect.append(ATE)
    
    @timer
    def GRF(self, SimData, estimates, treatment, grid, depth):
        # Specifying the model:
        grf_NL = CausalForest(criterion='het',
                           n_estimators=1000,
                           min_samples_leaf=5,
                           max_depth=depth,
                           min_impurity_decrease = 0.0,
                           max_samples=0.45,
                           warm_start=False,
                           inference=False,
                           fit_intercept=True,
                           subforest_size=4,
                           honest=True,
                           verbose=0,
                           n_jobs=-1)
        # Fitting:
        grf_NL.fit(y = SimData.y - SimData.ln_sjg * estimates.rho[-1] - SimData.X[:,1] * estimates.alpha[-1],
                   X = np.delete(SimData.X[:,2:5], treatment - 2, 1),
                   T = SimData.X[:, treatment].reshape(-1,1))
        # Grid of effects:
        CATE = grf_NL.predict(grid)
        self.effect.append(CATE)
    
    @timer
    def MLP(self, SimData, estimates, grid):
        # Loop to rerun the function if the weights diverge
        marginal_effects = np.array([np.nan])
        while np.sum(np.isnan(marginal_effects)) > 0:
            # Specifying the model:
            model = keras.Sequential()
            model.add(keras.layers.Dense(128, activation = 'relu'))
            model.add(keras.layers.Dense(64, activation = 'relu'))
            model.add(keras.layers.Dense(1))
            model.compile(optimizer = 'adam', loss = 'mean_squared_error')
            model.fit(SimData.X[:,2:5],
                      SimData.y - SimData.ln_sjg * estimates.rho[-1] - SimData.X[:,1] * estimates.alpha[-1],
                      epochs = 50,
                      validation_split = 0.1,
                      verbose = False,
                      callbacks = keras.callbacks.EarlyStopping(patience = 3, restore_best_weights = True))
            preds = model.predict(gridNN).reshape(-1)
            marginal_effects = np.diff(preds) * 10  # normalizing the gradient from a 0.1 change in x to 1
        self.effect.append(marginal_effects)

# 3. Monte Carlo Analysis

In [6]:
Lin_est    = FirstStepEstimates()
LinLog_est = FirstStepEstimates()
DMLIV_est  = FirstStepEstimates()
DML_est_x  = SecondStepEstimates()
DML_est_d  = SecondStepEstimates()
GRF_est_x  = SecondStepEstimates()
GRF_est_d  = SecondStepEstimates()
MLP_est    = SecondStepEstimates()

iterations = 2
iteration_time = []
# Specifying the grids on which to estimate the GRF and deep IV marginal effects
gridRFx = np.array([(x,y) for x in range(2) for y in np.linspace(0.1, 5, 50)])
gridRFd = np.array([(x,y) for x in np.linspace(0.1, 5, 50) for y in np.linspace(0.1, 5, 50)])
gridNN  = np.array([(x,y,z) for x in range(2) for y in np.arange(1, 6, 2) for z in np.arange(0.1, 5.1, 0.1)])

for i in range(iterations):
    iter_start = time.perf_counter()
    print(i + 1, end = ' ')
    
    # Simulating a new dataset
    dataset = LogitSim()
    
    # First step estimators
    Lin_est.LinIV(dataset)
    LinLog_est.LinIVLog(dataset)
    DMLIV_est.DMLIV(dataset,
                    depth_yx, n_est_yx, lr_yx,     # SET VALUES TO RUN
                    depth_tx, n_est_tx, lr_tx,     # SET VALUES TO RUN
                    depth_txz, n_est_txz, lr_txz)  # SET VALUES TO RUN
    
    # Second step estimators
    DML_est_x.DML(dataset, DMLIV_est, 3,  # treatment = x1
                  depth_y, n_est_y, lr_y,          # SET VALUES TO RUN
                  depth_t, n_est_t, lr_t)          # SET VALUES TO RUN
    DML_est_d.DML(dataset, DMLIV_est, 2,  # treatment = d
                  depth_y, n_est_y, lr_y,          # SET VALUES TO RUN
                  depth_t, n_est_t, lr_t)          # SET VALUES TO RUN
    GRF_est_x.GRF(dataset, DMLIV_est, 3,  # treatment = x1
                  gridRFx, depth)                  # SET VALUES TO RUN
    GRF_est_d.GRF(dataset, DMLIV_est, 2,  # treatment = d
                  gridRFd, depth)                  # SET VALUES TO RUN
    MLP_est.MLP(dataset, DMLIV_est, gridNN)
    
    iteration_time.append(time.perf_counter() - iter_start)
    print('| Runtime:', np.round(iteration_time[i], 2))

# 4. Saving Estimates

In [7]:
export_dir = f'Estimates {LogitSim.T}x{LogitSim.J}'
if not os.path.exists(export_dir):
    os.makedirs(export_dir)

# First step parameter estimates
pd.DataFrame(
    {
        'alpha_NL': Lin_est.alpha,
        'rho_NL': Lin_est.rho,
        'alpha_NL_log': LinLog_est.alpha,
        'rho_NL_log': LinLog_est.rho,
        'alpha_NL_XGB': DMLIV_est.alpha,
        'rho_NL_XGB': DMLIV_est.rho
    }
).to_csv(export_dir + '\\step1.csv', index = False)

# Second step linear coefficients/ATE estimates
pd.DataFrame(
    {
        'NL_theta_x': np.array(Lin_est.beta)[:,2],
        'NL_theta_d': np.array(Lin_est.beta)[:,1],
        'NL_theta_log_x': np.array(LinLog_est.beta)[:,2],
        'NL_theta_log_d': np.array(LinLog_est.beta)[:,1],
        'NL_XGB_theta_x': DML_est_x.effect,
        'NL_XGB_theta_d': DML_est_d.effect
    }
).to_csv(export_dir + '\\step2_theta.csv', index = False)

# Second step CATE estimates
pd.DataFrame(np.array(GRF_est_x.effect).reshape(-1, gridRFx.shape[0])).to_csv(export_dir + '\\step2_GRF_x.csv', index = False)
pd.DataFrame(np.array(GRF_est_d.effect).reshape(-1, gridRFd.shape[0])).to_csv(export_dir + '\\step2_GRF_d.csv', index = False)

# Second step marginal effects estimates
pd.DataFrame(np.array(MLP_est.effect).reshape(-1, gridNN.shape[0] - 1)).to_csv(export_dir + '\\step2_MLP.csv', index = False)

# Elasticity estimates RMSEs
pd.DataFrame(
    np.array(Lin_est.eta_summary)[~np.isinf(Lin_est.eta_summary).any(axis = 1), 3:6],
    columns = ['Own', 'Cross (same)', 'Cross (different)']
).to_csv(export_dir + '\\lin_RMSE.csv', index = False)
pd.DataFrame(
    np.array(LinLog_est.eta_summary)[~np.isinf(LinLog_est.eta_summary).any(axis = 1), 3:6],
    columns = ['Own', 'Cross (same)', 'Cross (different)']
).to_csv(export_dir + '\\log_RMSE.csv', index = False)
pd.DataFrame(
    np.array(DMLIV_est.eta_summary)[~np.isinf(DMLIV_est.eta_summary).any(axis = 1), 3:6],
    columns = ['Own', 'Cross (same)', 'Cross (different)']
).to_csv(export_dir + '\\xgb_RMSE.csv', index = False)